<a href="https://colab.research.google.com/github/zakbosson-oss/DSA210_Project/blob/main/DSA_210_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STEP 1: Gather Data From Channels Within The Niche

The initial step of the project is to gather data. The data will be self obtained with the usage of YouTube Data API v3, Yt-dlp and Youtube Transcript Scraper.


In [3]:
#@title 1.1 Libraries + API
!pip install google-api-python-client


from google.colab import userdata

API_KEY = userdata.get('API_KEY')

#Import Libraries
import googleapiclient.discovery


In [2]:
#@title 1.2 Search For Channels Function Defination
def search_channels(query, max_results=25, api_key=None):
    youtube = googleapiclient.discovery.build(
        "youtube", "v3", developerKey=api_key
    )

    request = youtube.search().list(
        q=query,
        part="snippet",
        type="channel",
        maxResults=max_results
    )

    response = request.execute()

    channels = []
    for item in response["items"]:
        channels.append({
            "channel_title": item["snippet"]["channelTitle"],
            "channel_id": item["snippet"]["channelId"],
            "description": item["snippet"]["description"]
        })

    return channels


In [3]:
#@title 1.3 Get Channel Statistics Function
def get_channel_statistics(channel_ids, api_key):
    """
    Takes a list of channel IDs and returns stats:
    subscriberCount, viewCount, videoCount.
    """
    youtube = googleapiclient.discovery.build(
        "youtube", "v3", developerKey= api_key
    )

    request = youtube.channels().list(
        part="statistics,snippet",
        id=",".join(channel_ids)
    )
    response = request.execute()

    stats = []

    for item in response["items"]:
        stats.append({
            "channel_id": item["id"],
            "channel_title": item["snippet"]["title"],
            "description": item["snippet"]["description"],
            "subscribers": int(item["statistics"].get("subscriberCount", 0)),
            "total_views": int(item["statistics"].get("viewCount", 0)),
            "video_count": int(item["statistics"].get("videoCount", 0))
        })

    return stats


In [1]:
#@title 1.4 Search For Channels With The Given Keywords

keywords = ["WW2", "World War 2", "WWII history", "World War Two"]

all_channels = []

# Step A: Search channels by keywords
for keyword in keywords:
    results = search_channels(keyword, max_results=30, api_key=API_KEY)
    all_channels.extend(results)

# Step B: Deduplicate by channel ID
unique_channels = {c["channel_id"]: c for c in all_channels}

# Step C: Fetch channel statistics
channel_ids = list(unique_channels.keys())


channel_stats = get_channel_statistics(channel_ids, API_KEY)

channel_stats

NameError: name 'search_channels' is not defined